# Web Analytics (2IID0) Homework Assignment 3
## Exceptional Model Ming vs A/B Testing
#### <i>Abdel K. Bokharouss, Bart van Helvert, Joris Rombouts & Remco Surtel</i>   -   December 2017

In this assignment we will combine Exceptional Model Mining (EMM) and A/B Testing. The dataset has been provided by StudyPortals. The core concept of A/B Testing is that each test subject gets a corresponding variant assigned. After that, we measure the rate of success per variant and the variant with most success is kept, while the other is discarded. However, subgroups that might be served better by the losing version will be disadvantaged. To do better than this, we further mine the data to find coherent subgroups where alternative delivers more success. New visitors to the website that correspond to a specific subgroup, will get either version A or B of the website. Exceptional Model Mining allows us to mine the data further, so that we can discover these subgroups. First, we need to make a decision which attributes will be used as descriptors and which of the attributes will be used as targets.

### <font color="green">imports, preparation and configuration</font>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import tree, preprocessing, metrics
import matplotlib.pyplot as plt
%matplotlib inline

### 1 Data Preprocessing

#### a
The raw data that StudyPortals delivered, contains of three datasets: `clicking_data`, `experiment_datails` and `meta_data`. The primary imporance to StudyPortals is analyzing the association between which version of the website were shown (version A or B) and whether or not the user clicked on the button. Therefore the binary attribute `action` of the dataset `clicking_data` is the first target attribute we define. However, we also want to know which version (A or B) was showed to the user. Therefore the attribute `condition` of the dataset `experiment_details` is the second target attribute we define.

#### b
After the targets are defined, the next step to do is to select which attributes will be used as descriptors. The goal is to gather as much descriptor informaiton as is reasonably possible. First we explore the data, and check how many NAN-values the three datasets contain.

In [2]:
df_click = pd.read_csv('data/clicking_data.csv')
df_click.head()

,action,action_label,action_type,tstamp,user_session
0,clic,revenue,link,1472755490,379881d5-32d7-49f4-bf5b-81fefbc5fcce
1,clic,revenue,link,1472839117,2a0f4218-4f62-479b-845c-109b2720e6e7
2,clic,revenue,link,1472879219,a511b6dc-2dca-455b-b5e2-bf2d224a5505
3,clic,revenue,link,1472890876,9fb616a7-4e13-4307-ac92-0b075d7d376a
4,clic,revenue,link,1472892380,64816772-688d-4460-a591-79aa49bba0d5


In [3]:
print(df_click.shape)
df_click.info()

(919, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 5 columns):
action          919 non-null object
action_label    919 non-null object
action_type     919 non-null object
tstamp          919 non-null int64
user_session    919 non-null object
dtypes: int64(1), object(4)
memory usage: 36.0+ KB


In [4]:
df_experiment = pd.read_csv('data/experiment_details.csv')
df_experiment.head()

,user_id,experiment_id,condition,timestamp
0,7f693927-6835-415e-b817-794eabee96d2,86-v2-Butonny buttons,2-Buttony-Conversion-Buttons,16:59.2
1,8c07c192-83ab-4646-8cb5-b3a692e6eedf,86-v2-Butonny buttons,2-Buttony-Conversion-Buttons,17:10.3
2,6aab759e-5662-4578-9ad5-016895b3a52b,86-v2-Butonny buttons,2-Buttony-Conversion-Buttons,20:39.9
3,0b31f493-2a44-4e9c-ac76-a30cd9d844f0,86-v2-Butonny buttons,2-Buttony-Conversion-Buttons,21:12.6
4,e502aa88-8429-4981-80c9-526bb190e76f,86-v2-Butonny buttons,1-Control,30:02.3


In [5]:
print(df_experiment.shape)
df_experiment.info()

(14906, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14906 entries, 0 to 14905
Data columns (total 4 columns):
user_id          14906 non-null object
experiment_id    14906 non-null object
condition        14906 non-null object
timestamp        14906 non-null object
dtypes: object(4)
memory usage: 465.9+ KB


In [6]:
df_meta = pd.read_csv('data/meta_data_modified.csv', encoding = 'ansi')
df_meta.head()

,platform,etl_tstamp,collector_tstamp,dvce_created_tstamp,event,domain_userid,domain_sessionid,user_id,geo_country,geo_region,...,pp_yoffset_max,useragent,browser_language,browser_cookies,browser_colordepth,browser_viewdepth,browser_viewheight,os_name,os_timezone,dvce_type
0,web,06:22.2,06:33.7,06:26.2,page_ping,e8702985-ffee-4ede-a89f-396312539812,0ca47d52-1375-4346-ab2e-b205b64f642e,bc5effb7-a476-414f-a62e-5479022e7553,GR,,...,915.0,Mozilla/5.0 (Linux; Android 4.4.4; GT-I9300 Bu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,web,06:49.2,07:00.2,06:52.6,page_ping,e8702985-ffee-4ede-a89f-396312539812,0ca47d52-1375-4346-ab2e-b205b64f642e,bc5effb7-a476-414f-a62e-5479022e7553,GR,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,web,07:09.2,07:20.1,07:12.6,page_ping,e8702985-ffee-4ede-a89f-396312539812,0ca47d52-1375-4346-ab2e-b205b64f642e,bc5effb7-a476-414f-a62e-5479022e7553,GR,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,web,09:49.3,10:01.0,09:53.4,page_ping,e8702985-ffee-4ede-a89f-396312539812,0ca47d52-1375-4346-ab2e-b205b64f642e,bc5effb7-a476-414f-a62e-5479022e7553,GR,,...,3411.0,Mozilla/5.0 (Linux; Android 4.4.4; GT-I9300 Bu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,web,10:40.3,10:51.8,10:44.4,page_ping,e8702985-ffee-4ede-a89f-396312539812,0ca47d52-1375-4346-ab2e-b205b64f642e,bc5effb7-a476-414f-a62e-5479022e7553,GR,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
print(df_meta.shape)
df_meta.info()

(171949, 30)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171949 entries, 0 to 171948
Data columns (total 30 columns):
platform               171949 non-null object
etl_tstamp             171949 non-null object
collector_tstamp       171949 non-null object
dvce_created_tstamp    171949 non-null object
event                  171949 non-null object
domain_userid          171949 non-null object
domain_sessionid       171949 non-null object
user_id                171949 non-null object
geo_country            171690 non-null object
geo_region             171949 non-null object
geo_city               88981 non-null object
geo_region_name        87899 non-null object
geo_timezone           165463 non-null object
page_url               171949 non-null object
page_title             133368 non-null object
page_referrer          63876 non-null object
refr_source            26349 non-null object
pp_xoffset_min         40918 non-null float64
pp_xoffset_max         40918 non-null float64
pp_yof

In [8]:
sum(df_meta.apply(lambda x: sum(x.isnull().values), axis = 1) > 0) # number of rows with NaN values

171862

### 2 A&B Testing

### 3 Beyond A&B Testing